# Изградња регресионог модела: припрема и визуализација података

## **Линеарна регресија за бундеве - Лекција 2**
#### Увод

Сада када сте опремљени алатима који су вам потребни за почетак изградње модела машинског учења уз помоћ Tidymodels и Tidyverse-а, спремни сте да почнете постављати питања о вашим подацима. Док радите са подацима и примењујете решења машинског учења, веома је важно разумети како поставити право питање како бисте на прави начин искористили потенцијале вашег скупа података.

У овој лекцији ћете научити:

- Како припремити податке за изградњу модела.

- Како користити `ggplot2` за визуализацију података.

Питање на које желите одговор одредиће који тип алгоритама машинског учења ћете користити. Квалитет одговора који добијете у великој мери ће зависити од природе ваших података.

Хајде да ово видимо кроз практичну вежбу.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Илустрација: @allison_horst</figcaption>


<!--![Илустрација: \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Илустрација: \@allison_horst-->


## 1. Увоз података о бундевама и позивање Tidyverse-а

Биће нам потребни следећи пакети за обраду и анализу података у овој лекцији:

-   `tidyverse`: [Tidyverse](https://www.tidyverse.org/) је [збирка R пакета](https://www.tidyverse.org/packages) осмишљена да учини науку о подацима бржом, једноставнијом и забавнијом!

Можете их инсталирати на следећи начин:

`install.packages(c("tidyverse"))`

Скрипта испод проверава да ли имате инсталиране пакете потребне за завршетак овог модула и инсталира их уколико неки недостају.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Сада, хајде да покренемо неке пакете и учитамо [подаци](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) који су обезбеђени за ову лекцију!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Брзи `glimpse()` одмах показује да постоје празнине и мешавина стрингова (`chr`) и нумеричких података (`dbl`). `Date` је типа карактер, а ту је и чудна колона названа `Package` где су подаци мешавина између `sacks`, `bins` и других вредности. Подаци су, у ствари, прилично неуредни 😤.

Заправо, није баш уобичајено добити скуп података који је потпуно спреман за употребу и креирање ML модела одмах. Али не брините, у овој лекцији ћете научити како да припремите необрађени скуп података користећи стандардне R библиотеке 🧑‍🔧. Такође ћете научити различите технике за визуализацију података.📈📊
<br>

> Подсетник: Оператор цеви (`%>%`) извршава операције у логичком низу тако што прослеђује објекат функцији или изразу. Можете замислити оператор цеви као да у вашем коду каже "и онда".


## 2. Провера недостајућих података

Један од најчешћих проблема са којима се научници за податке сусрећу јесте непотпуни или недостајући подаци. У R-у се недостајуће или непознате вредности представљају посебном вредношћу: `NA` (Not Available).

Како бисмо знали да ли оквир података садржи недостајуће вредности?  
<br>
- Један једноставан начин био би коришћење основне R функције `anyNA`, која враћа логичке вредности `TRUE` или `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Одлично, чини се да недостају неки подаци! То је добро место за почетак.

-   Други начин би био да се користи функција `is.na()` која указује на то који појединачни елементи у колони недостају логичком вредношћу `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

У реду, завршен посао, али са овако великим оквиром података, било би неефикасно и практично немогуће прегледати све редове и колоне појединачно😴.

-   Интуитивнији начин био би да израчунате збир недостајућих вредности за сваку колону:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Много боље! Недостају неки подаци, али можда то неће бити важно за задатак који је пред нама. Да видимо шта ће даља анализа донети.

> Поред сјајних сетова пакета и функција, R има веома добру документацију. На пример, користите `help(colSums)` или `?colSums` да бисте сазнали више о функцији.


## 3. Dplyr: Граматика за манипулацију подацима

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Илустрација: @allison_horst</figcaption>


<!--![Илустрација: \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Илустрација: \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), пакет у оквиру Tidyverse-а, представља граматику за манипулацију подацима која пружа конзистентан сет глагола који вам помажу да решите најчешће изазове у манипулацији подацима. У овом делу, истражићемо неке од глагола из dplyr-а!
<br>


#### dplyr::select()

`select()` је функција у пакету `dplyr` која вам помаже да изаберете колоне које желите да задржите или изузмете.

Да бисте свој рад са подацима учинили једноставнијим, уклоните неколико колона из вашег оквира података користећи `select()`, задржавајући само оне колоне које су вам потребне.

На пример, у овој вежби, наша анализа ће обухватати колоне `Package`, `Low Price`, `High Price` и `Date`. Хајде да изаберемо те колоне.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` је функција у пакету `dplyr` која вам помаже да креирате или модификујете колоне, а да притом задржите постојеће колоне.

Општа структура функције `mutate` је:

`data %>%   mutate(new_column_name = what_it_contains)`

Хајде да испробамо `mutate` користећи колону `Date` и извршимо следеће операције:

1. Претворимо датуме (који су тренутно типа карактер) у формат месеца (ово су датуми у америчком формату, дакле `MM/DD/YYYY`).

2. Извуцимо месец из датума у нову колону.

У програмском језику R, пакет [lubridate](https://lubridate.tidyverse.org/) олакшава рад са подацима типа датум-време. Дакле, хајде да користимо `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` и видимо како да постигнемо горе наведене циљеве. Можемо избацити колону `Date` јер нам више неће бити потребна у наредним операцијама.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Ура! 🤩

Следеће, хајде да направимо нову колону `Price`, која представља просечну цену бундеве. Сада, хајде да узмемо просек из колона `Low Price` и `High Price` како бисмо попунили нову колону Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Дааа!💪

"Али чекај!", рећи ћеш након што прелетиш цео скуп података помоћу `View(pumpkins)`, "Овде нешто није у реду!"🤔

Ако погледаш колону `Package`, бундеве се продају у различитим конфигурацијама. Неке се продају у мерама `1 1/9 бушел`, неке у мерама `1/2 бушел`, неке по бундеви, неке по фунти, а неке у великим кутијама различитих ширина.

Хајде да ово проверимо:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Невероватно!👏

Чини се да је веома тешко доследно измерити бундеве, па хајде да их филтрирамо тако што ћемо изабрати само бундеве које у колони `Package` садрже реч *bushel* и ставимо их у нови податак оквира `new_pumpkins`.


#### dplyr::filter() и stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): креира подскуп података који садржи само **редове** који испуњавају ваше услове, у овом случају, тикве са низом *bushel* у колони `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): открива присуство или одсуство шаблона у низу.

Пакет [`stringr`](https://github.com/tidyverse/stringr) пружа једноставне функције за уобичајене операције са низовима.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Можете видети да смо сузили избор на око 415 редова података који садрже бундеве по бушелу.🤩  
<br>


#### dplyr::case_when()

**Али сачекајте! Постоји још нешто што треба урадити**

Да ли сте приметили да количина у бушелима варира по реду? Потребно је нормализовати цене тако да приказујете цену по бушелу, а не по 1 1/9 или 1/2 бушела. Време је за мало математике како бисмо то стандардизовали.

Користићемо функцију [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) да *изменимо* колону Price у зависности од одређених услова. `case_when` нам омогућава да векторизујемо више `if_else()` изјава.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Сада можемо анализирати цену по јединици на основу мерења по бушелу. Све ово проучавање бушела бундева, међутим, показује колико је `важно` да `разумете природу ваших података`!

> ✅ Према [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), тежина бушела зависи од врсте производа, јер је то мера запремине. "Бушел парадајза, на пример, треба да тежи 56 фунти... Листови и зелено поврће заузимају више простора са мање тежине, па бушел спанаћа тежи само 20 фунти." Све је то прилично компликовано! Нећемо се мучити са конверзијом бушела у фунте, већ ћемо одредити цену по бушелу. Све ово проучавање бушела бундева, међутим, показује колико је важно да разумете природу ваших података!
>
> ✅ Да ли сте приметили да су бундеве које се продају по пола бушела веома скупе? Можете ли схватити зашто? Савет: мале бундеве су много скупље од великих, вероватно зато што их има много више по бушелу, с обзиром на неискоришћени простор који заузима једна велика шупља бундева за питу.


Сада, зарад авантуре 💁‍♀️, хајде да преместимо колону "Месец" на прву позицију, односно `пре` колоне `Пакет`.

`dplyr::relocate()` се користи за промену позиција колона.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Сјајно!👌 Сада имате чист и уредан скуп података на коме можете изградити свој нови регресиони модел!  
<br>


## 4. Визуелизација података са ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Инфографика од Дасани Мадипали</figcaption>


<!--![Инфографика од Дасани Мадипали](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Постоји једна *мудра* изрека која гласи:

> "Једноставан графикон је донео више информација у ум аналитичара података него било који други уређај." --- Џон Туки

Део улоге научника за податке је да демонстрира квалитет и природу података са којима ради. Да би то урадили, често креирају занимљиве визуелизације, или графиконе, дијаграме и табеле, које приказују различите аспекте података. На овај начин могу визуелно приказати односе и празнине које је иначе тешко открити.

Визуелизације такође могу помоћи у одређивању технике машинског учења која је најприкладнија за податке. На пример, расејани графикон који изгледа као да прати линију указује на то да су подаци добар кандидат за вежбу линеарне регресије.

R нуди неколико система за креирање графикона, али [`ggplot2`](https://ggplot2.tidyverse.org/index.html) је један од најелегантнијих и најсвестранијих. `ggplot2` вам омогућава да саставите графиконе **комбинујући независне компоненте**.

Хајде да почнемо са једноставним расејаним графиконом за колоне Price и Month.

У овом случају, почећемо са [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), додаћемо скуп података и естетско мапирање (са [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), а затим додати слојеве (као [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) за расејане графиконе.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Да ли је ово користан графикон 🤷? Да ли вас нешто на њему изненађује?

Није нарочито користан јер све што ради јесте да приказује ваше податке као распоред тачака у датом месецу.  
<br>


### **Како да учинимо да буде корисно?**

Да би графикони приказивали корисне податке, обично је потребно некако груписати податке. На пример, у нашем случају, проналажење просечне цене бундева за сваки месец би пружило више увида у основне обрасце у нашим подацима. Ово нас доводи до још једног брзог прегледа функција пакета **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Груписана агрегација у R-у може се лако израчунати помоћу

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` мења јединицу анализе са целокупног скупа података на појединачне групе, као што су групе по месецима.

-   `dplyr::summarize()` креира нови скуп података са једном колоном за сваку променљиву груписања и једном колоном за сваку од статистика које сте навели.

На пример, можемо користити `dplyr::group_by() %>% summarize()` да групишемо бундеве у групе на основу колоне **Month** и затим израчунамо **просечну цену** за сваки месец.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Сажето!✨

Категоријске карактеристике, као што су месеци, боље се представљају помоћу стубичастог графикона 📊. Слојеви који су одговорни за стубичасте графиконе су `geom_bar()` и `geom_col()`. Погледајте `?geom_bar` за више информација.

Хајде да направимо један!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Ово је кориснија визуализација података! Чини се да указује на то да се највиша цена бундева јавља у септембру и октобру. Да ли то одговара вашим очекивањима? Зашто или зашто не?

Честитамо на завршетку друге лекције 👏! Припремили сте своје податке за изградњу модела, а затим открили више увида користећи визуализације!



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
